In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

In [16]:
df = pd.read_csv('combined_dataset.csv')

df.rename(columns={'Unnamed: 0': 'Local Date'}, inplace=True)
df['Unnamed: 0'] = pd.to_datetime(df['Local Date'])
df = df.sort_values(by='Local Date')

# Assuming the dataset spans a considerable time range, we can pick a cut-off date
# For example, if we want to use the last 20% of dates for testing, we can do the following:
cut_off_index = int(len(df) * 0.8)
cut_off_date = df.iloc[cut_off_index]['Local Date']

# Splitting the data based on the cut-off date
train_df = df[df['Local Date'] < cut_off_date]
test_df = df[df['Local Date'] >= cut_off_date]

# Preparing the feature (X) and target (y) variables
X_train = train_df.drop(columns=['NASDAQ_Volatility', 'Local Date', 'Unnamed: 0']).values
y_train = train_df['NASDAQ_Volatility'].values
X_test = test_df.drop(columns=['NASDAQ_Volatility', 'Local Date', 'Unnamed: 0']).values
y_test = test_df['NASDAQ_Volatility'].values


In [17]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30],
}

# Initialize the grid search model
grid_search = GridSearchCV(estimator=RandomForestRegressor(random_state=42), 
                           param_grid=param_grid, 
                           cv=3, 
                           n_jobs=-1, 
                           verbose=2)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Get the best parameters
best_params = grid_search.best_params_
best_params

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] END .....................max_depth=10, n_estimators=100; total time=  50.5s
[CV] END .....................max_depth=10, n_estimators=100; total time=  50.7s
[CV] END .....................max_depth=10, n_estimators=100; total time=  51.0s
[CV] END .....................max_depth=20, n_estimators=100; total time= 1.0min
[CV] END .....................max_depth=10, n_estimators=200; total time= 1.7min
[CV] END .....................max_depth=10, n_estimators=200; total time= 1.7min
[CV] END .....................max_depth=10, n_estimators=200; total time= 1.7min
[CV] END .....................max_depth=20, n_estimators=100; total time=  59.3s
[CV] END .....................max_depth=20, n_estimators=100; total time= 1.0min
[CV] END .....................max_depth=10, n_estimators=300; total time= 2.5min
[CV] END .....................max_depth=10, n_estimators=300; total time= 2.5min
[CV] END .....................max_depth=10, n_est

{'max_depth': 30, 'n_estimators': 200}

In [18]:
y_pred = grid_search.predict(X_test)

In [19]:
r_squared = r2_score(y_test, y_pred)
print(f'R-squared on Test Set: {r_squared}')


R-squared on Test Set: -0.22822437555399988
